In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize


# In[25]:


#Ecrire les équations du modèle direct 𝑋=F(theta), puis du résidu 𝑅(theta) pour une position d’outil donnée.
def F(theta1,theta2):#Modème directe
    X=[l1*np.cos(theta1)+l2*np.cos(theta1+theta2),l1*np.sin(theta1)+l2*np.sin(theta1+theta2)]
    return X


# In[26]:


def fun(x):#fontion de résidus
    return [l1*np.cos(x[0])  + l2*np.cos(x[0]+x[1])-P[0],
            l1*np.sin(x[0])  + l2*np.sin(x[0]+x[1])-P[1]]
def jac(x):#jacobien de résidus
    return np.array([[-l1*np.sin(x[0])-l2*np.sin(x[0]+x[1]),
                      -l2*np.sin(x[0]+x[1])],
                     [l1*np.cos(x[0])+l2*np.cos(x[0]+x[1]),
                      l2*np.cos(x[0]+x[1])]])
def f(x):#carre_norme_residus
    return (l1*np.cos(x[0])  + l2*np.cos(x[0]+x[1])-P[0])**2+(l1*np.sin(x[0])  + l2*np.sin(x[0]+x[1])-P[1])**2
def j(theta1,theta2):#jaobien de carre_norme_residus
    r1=2*(l1*np.cos(theta1)+l2*np.cos(theta1+theta2)-P[0])*(-l1*np.sin(theta1)-l2*np.sin(theta1+theta2))+2*(l1*np.sin(theta1)+l2*np.sin(theta1+theta2)-P[1])*(l1*np.cos(theta1)+l2*np.cos(theta1+theta2))
    r2=2*(l1*np.cos(theta1)+l2*np.cos(theta1+theta2)-P[0])*(-l2*np.sin(theta1+theta2))+2*(l1*np.sin(theta1)+l2*np.sin(theta1+theta2)-P[1])*(l2*np.cos(theta1+theta2))
    return r1,r2
def H(theta1,theta2):#Hessian de carre_norme_residus
    h1=2*P[0]*(l1*np.cos(theta1)+l2*np.cos(theta1+theta2))+ 2*P[1]*(l1*np.sin(theta1)+l2*np.sin(theta1+theta2))
    h2=2*P[0]*l2*np.cos(theta1+theta2)+2*P[1]*l2*np.sin(theta1+theta2)
    h3=2*P[0]*l2*np.cos(theta1+theta2)+2*P[1]*l2*np.sin(theta1+theta2)
    h4=2*(l1*np.cos(theta1)-P[0])*(-l2*np.cos(theta1+theta2))+ 2*(l1*np.sin(theta1)-P[1])*(-l2*np.sin(theta1+theta2))
    return h1,h2,h3,h4


# In[27]:


def p(angle,l):#exprimer le vecteur d'un segment avec un variable pi qui est l'anglre entre le vecteur OP et l'axe x et un variable de la module de ce vecteurm,cette fontion sera utilisée dans la fonction solution_trajectoire qui nous permet de tracer le trajectoire des deux bras d'une ou plusieur solutions
    return [l*np.cos(angle),l*np.sin(angle)]
def f1(x):#cette fonction sera utilisé dans la fonction solution_analytique,il nous petmet de calculer le coordonnée d'un intersection des deux ligne 
    return [(l1*np.cos(theta[0])-x[0])**2+(l1*np.sin(theta[0])-(P[1]/P[0])*x[0])**2-((l1*l2*np.sin(theta[1]))/(P[0]**2+P[1]**2))**2]
def jac_f1(x):#jacobien de la fontion f1
    return np.array([-2*(l1*np.cos(theta[0])-x[0])-2*(P[1]/P[0])*(l1*np.sin(theta[0])-(P[1]/P[0])*x[0])])   


# In[28]:
def Newton(theta,precision,nmax) :#méthode newton avec trois vrriables(theta:le point départ choisi,précision:la précision d'arret, nmax:le nombre max d'itération)
    global r1,r2,h1,h2,h3,h4#le varriable theta est le point départ 
    theta1=theta[0]
    theta2=theta[1]
    tab_theta1=[theta1]#comme méthode gradient, on cree deux list, ils seront utulisés dans la fontion résultat_iteration_newton qui permet de s'afficher le résultat d'itetation pour un point départ choisi dans la graphe 
    tab_theta2=[theta2]
    h1,h2,h3,h4=H(theta1,theta2)#les élément de la matrice héssian
    r1,r2=j(theta1,theta2)#les éléments de la matrice jacobienne
    dX=1#initialisation 
    n=0
    while dX >= precision and n <= nmax : 
        delta_theta1=(r2*h2-r1*h4)/(h1*h4-h2*h3)#calculer le delta_theta par la formule H_residus(theta1,theta2)*delta_theta=-J_résidus(theta1,theta2)
        delta_theta2=(r2*h1-r1*h3)/(h2*h3-h1*h4)
        theta1+=delta_theta1#renouvellement de theta1 et theta2
        theta2+=delta_theta2
        tab_theta1.append(theta1)
        tab_theta2.append(theta2)
        h1,h2,h3,h4=H(theta1,theta2)#renoubellement de la matrice hessian et jacobien
        r1,r2=j(theta1,theta2)
        dX=np.sqrt((delta_theta1)**2+(delta_theta2)**2)#renouvellement de dX
        n += 1
        
    return [theta1,theta2],tab_theta1,tab_theta2

def résultat_iteration_newton(X,l,theta,precision,nmax):#cette fonction est pour tracer le resultat d'iteration de la methode newton a partir d'un point départ choisi dans le graphe 
    global P                                            #le varriable theta est le point départ 
    global l1,l2
    l1=l[0]
    l2=l[1]
    P=[X[0],X[1]]
    theta1min, theta1max, ntheta1 = -10 ,10, 100   
    theta2min, theta2max, ntheta2 = -10, 10, 100
    theta1d = np.linspace(theta1min,theta1max,ntheta1)
    theta2d = np.linspace(theta2min,theta2max,ntheta2)
    Theta1d, Theta2d = np.meshgrid(theta1d, theta2d) #créer la maillage 
    nIso = 10
    plt.figure(figsize=(40, 40))
    plt.subplot(121)
    cp = plt.contour(Theta1d,Theta2d,f([Theta1d,Theta2d]),nIso)#tracer le graphe de isovaleur pour la fontion résidus
    plt.clabel(cp, inline=True,fontsize=10)
    plt.title("le résultat d'itération avec le point départ choisi par la méthode newton")
    plt.xlabel('theta1')
    plt.ylabel('theta2')
    plt.grid()
    plt.axis('square')
    plt.scatter(Newton(theta,precision,nmax)[1],Newton(theta,precision,nmax)[2])#tracer le résultat d'itération de la méthode newton 
    plt.plot(Newton(theta,precision,nmax)[1][0],Newton(theta,precision,nmax)[2][0],'.r',label="point départ") # Départ
    plt.plot(Newton(theta,precision,nmax)[1][-1],Newton(theta,precision,nmax)[2][-1],'.y',label="point arrivée") # Arrivée
    plt.legend()
    plt.show()

In [3]:
def solution_articulaire_4eme(X,l,precision,nmax):
    #définir les varriables glaobaux
    global P#la pose donné
    global l1,l2#les longuers des deux bras
    global Precision#le précision de la méthode newton
    global Nmax
    Precision=precision#choix de la précion 
    Nmax=nmax#choix du nombre max d'itération 
    l1=l[0]
    l2=l[1]
    P=[X[0],X[1]]
    indice=[]
    theta=[]#le list theta va etre rempli par [theta1,theta2] d'apres les résutats
    F_theta=[]#pour chaque solition trouvé([theta1,theta2]), on va calculer sa valeur f([theta1,theta2])(résidus), et on les met dans le list F_theta, ils seront utilisé dans critière de filtrage suivante
    valeur_bool1=True# la valeur booleene, il sera utilsée pour distinguer les cas l1<=l2 ou l1>l2, si l1<=l2-->l'espace atteignable est composé par une cercle
                     #si l1>l2--->l'espace atteignable est composé par l'espace entre le cercle extérieur et le cercle intérieur
    indice_point_depart=0
    #les point départ différent qui va etre testé pour trouver les solutions possibles, ici on choisi 400 points
    point_depart=np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.3306939635357677, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.6613879270715354, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 0.992081890607303, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.3227758541430708, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.6534698176788385, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 1.984163781214606, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.3148577447503738, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.6455517082861415, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 2.9762456718219092, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.306939635357677, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.6376335988934447, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 3.968327562429212, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.29902152596498, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.6297154895007475, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 4.960409453036515, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.291103416572283, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.621797380108051, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 5.9524913436438185, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586, 6.283185307179586]
    ,[0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586, 0.0, 0.3306939635357677, 0.6613879270715354, 0.992081890607303, 1.3227758541430708, 1.6534698176788385, 1.984163781214606, 2.3148577447503738, 2.6455517082861415, 2.9762456718219092, 3.306939635357677, 3.6376335988934447, 3.968327562429212, 4.29902152596498, 4.6297154895007475, 4.960409453036515, 5.291103416572283, 5.621797380108051, 5.9524913436438185, 6.283185307179586]])
        
    print("la methode de newton avec la précision choisi--->",Precision,"et le nombre d'itération max--->",Nmax)
    print("\nveuillez patienter, les donneés en train de chargement, la compilation peut prendre jusqu'a une minute")
    if l2<l1:valeur_bool1=False
    if l2>=l1:valeur_boo1=True
    for i in range(len(point_depart[0])):
        sol = Newton([point_depart[0][i],point_depart[1][i]],Precision,Nmax)[0]#pour chaque point départ, on le test avec la méthode de newton
        a=float('%.6f'%(sol[0]%6.283185))#on met les résultat de root en format de float et avec une précision du calcul de 1e-6 et on applique la minisation des valeur articumaire aussi
        b=float('%.6f'%(sol[1]%6.283185))
        indice.append([indice_point_depart])#on ajoute la indice point départ des résultat dans le list indice, il sera utilisé pour touver la point départ d'une solition a la fin 
        indice_point_depart+=1
        theta.append([a,b])#on ajoute les résultat de gradient dans un list 
        F_theta.append(np.sqrt(f([a,b])))#on ajoute le raccine de residus pour chaque point départ dans un list
    for i in range (len(F_theta)-1):#Nous trions les résultats de la valeur de residus  du plus petit au plus grand 
        for j in range (len(F_theta)-1-i):
            if F_theta[j]>F_theta[j+1]:
                F_theta[j],F_theta[j+1]=F_theta[j+1],F_theta[j]
                theta[j],theta[j+1]=theta[j+1],theta[j]
                indice[j],indice[j+1]=indice[j+1],indice[j]
    choix_affiche=input("veuillez taper 'yes' si vous vouler afficher les resultats de methode newton,sinon taper'no'")
    if choix_affiche=="yes":
        print("\nles résultats résolu par methode newton en 400 points départ:\n\ntheta----->\n",theta,"\n\nracinne du residus possible----->\n",F_theta)
    print("\n\npour la pose--->",P,"et les longuer des bras--->",l,"  les resultats d'analyse finale par methode gradient sont:")
    if F_theta[0]>=1e-4:#le valeur minimum de residus obtenur par newton est trop grand en ce cas, ca veux dire il n'y a pas de solution, on prend racinne de la valeur minimum de residu comme la distanc le plus court, et le theta correspondant comme la solution trouvée
        print("\nle cas pas de solution,le point le plus proche:","point depart(",[float(point_depart[0][indice[0]]),float(point_depart[1][indice[0]])],")   --->theta:",theta[0],"\net la distance le plus court:",F_theta[0])
        résultat_iteration_newton(X,l,[float(point_depart[0][indice[0]]),float(point_depart[1][indice[0]])],Precision,Nmax)#tracer le resultat d'iteration pour la solution trouvée dans un graphe
        point_depart=[[float(point_depart[0][indice[0]]),float(point_depart[1][indice[0]])]]
    elif valeur_bool1==True and (P[0]**2+P[1]**2)==(l1+l2)**2:#solution unique, ici, comme le cas pas de solution, on prends la valeur min de la racinne du residus, et le theta correspondant comme solution trouvée
        print("\nle cas solution unique,","point depart(",[float(point_depart[0][indice[0]]),float(point_depart[1][indice[0]])],")   --->theta:",theta[0],"\nvérification par modele geometrique:",F(theta[0][0],theta[0][1]))
        résultat_iteration_newton(X,l,[float(point_depart[0][indice[0]]),float(point_depart[1][indice[0]])],Precision,Nmax)
        point_depart=[[float(point_depart[0][indice[0]]),float(point_depart[1][indice[0]])]]
    elif valeur_bool1==False and ((P[0]**2+P[1]**2)==(l1+l2)**2 or (P[0]**2+P[1]**2)==(l1-l2)**2):
        print("\nle cas unique solution,","point depart(",[float(point_depart[0][indice[0]]),float(point_depart[1][indice[0]])],")   --->theta:",theta[0],"\nvérification par modele geometrique:",F(theta[0][0],theta[0][1]))
        résultat_iteration_newton(X,l,[float(point_depart[0][indice[0]]),float(point_depart[1][indice[0]])],Precision,Nmax)
        point_depart=[[float(point_depart[0][indice[0]]),float(point_depart[1][indice[0]])]]
    else:#le cas pas de solution, on sait bien il y a deux soltions possibles dans les resultat trouvés par newton en 400 point départ, on doit filtrer ces resultat, enlever les resultat équivalent, et on va trouver deux solution différent trouvées par gradient
        for i in range(len(theta)):#si theta1 theta2 sont égale deux pi ou moins deux pi, il est équibalent a la angle articulaire zéro
            if abs(theta[i][0]-6.28)<0.1 or abs(theta[i][0]+6.28)<0.1 or abs(theta[i][0])<0.1:
                theta[i][0]=0
            elif abs(theta[i][1]-6.28)<0.1 or abs(theta[i][1]+6.28)<0.1 or abs(theta[i][1])<0.1:
                theta[i][1]=0
        i=0
        j=1
        while i<len(theta):#on essaie de enlever les valeur équivalents 
            while j<len(theta):
                if abs(theta[i][0]-theta[j][0])<0.1 and abs(theta[i][1]-theta[j][1])<0.1:
                    theta.pop(j)
                    F_theta.pop(j)
                    indice.pop(j)
                else:
                    j+=1 
            i+=1
            j=i+1 
        print("\napres de le filtrage des valeurs équivalents---->\n",theta)
        #on prends les deux première solutions apres le filtrage comme les deux solutions finales
        for i in range(len(theta)-2):
                theta.pop(2)
                F_theta.pop(2)
                indice.pop(2)
        print("\nles solutions posisbles trouvées--->",theta,"\n")
        print("solution non_uniques!")
        print("premiere solution nous donne--->","point depart(",[float(point_depart[0][indice[0]]),float(point_depart[1][indice[0]])],")--->",theta[0],"  vérification par modèle geometrique--->",F(theta[0][0],theta[0][1]))
        résultat_iteration_newton(X,l,[float(point_depart[0][indice[0]]),float(point_depart[1][indice[0]])],Precision,Nmax)
        print("deuxieme solution nous donne--->","point depart(",[float(point_depart[0][indice[1]]),float(point_depart[1][indice[1]])],")--->",theta[1],"  vérification par modèle geometrique--->",F(theta[1][0],theta[1][1]))
        résultat_iteration_newton(X,l,[float(point_depart[0][indice[1]]),float(point_depart[1][indice[1]])],Precision,Nmax)     
        point_depart=[[float(point_depart[0][indice[0]]),float(point_depart[1][indice[0]])],[float(point_depart[0][indice[1]]),float(point_depart[1][indice[1]])]]
        
    return point_depart


# point_depart=solution_articulaire_3eme([3,4],[2,3],0.01,1000)
# print(point_depart[0])
# x,tab_x,_tab_y=gradient_pas_fix(list(point_depart)[0],0.01,1000)
# print(type(tab_x))

In [6]:
def corrdonnee_point(theta1,theta2):#cette sera utilisée dans la fontion solution_et_trajectoire
    return [[l1*np.cos(theta1),l1*np.sin(theta1)],[l1*np.cos(theta1)+l2*np.cos(theta1+theta2),l1*np.sin(theta1)+l2*np.sin(theta1+theta2)]]
def cercle1(x):#définir l'espace atteignable
    return np.sqrt((l1+l2)**2-x**2)
def cercle2(x):
    return -np.sqrt((l1+l2)**2-x**2)
%matplotlib qt5
def solution_et_trajectoire_avec_resultat_iteration(X,l,precision,nmax):
    #le temps t est défini dans la fontion solution_articulaire_1ere----50
    l1=l[0]
    l2=l[1]
    point_depart=solution_articulaire_4eme(X,l,precision,nmax)#en donnant les parametre X,L,alpha,nmax,solution_articulaire_3eme va nous donner une ou deux point points départ, on va utiliser ces point pour refaire l'iteration du gradient
    
    plt.figure(figsize=(10, 10))
    plt.ion()
    if len(point_depart)==1 :#unique solution ou pas de solution
        [theta1,theta2],tab_x,tab_y=Newton(point_depart[0],precision,nmax)
        for i in range(len(tab_x)):
            tabX = np.linspace(-l1-l2,l1+l2,2000)
            tabY = cercle1(tabX)
            tabY1= cercle2(tabX)
            plt.cla() #enlever l'inamge derniere
            plt.title("la position du bras articulé à différents instants avec le temps:10secondes\nunique solution possible")
            plt.grid()
            plt.xlabel("x")
            plt.xlim(-l1-l2-1, l1+l2+1)
            plt.xticks(np.linspace(-l1-l2-5, l1+l2+5, 15, endpoint=True))
            plt.ylabel("y")
            plt.ylim(-l1-l2-1, l1+l2+1)
            plt.yticks(np.linspace(-l1-l2-5, l1+l2+5, 15, endpoint=True))
        
            plt.plot([0,corrdonnee_point(tab_x[i],tab_y[i])[0][0]],[0, corrdonnee_point(tab_x[i],tab_y[i])[0][1]], c='r')
            plt.plot([corrdonnee_point(tab_x[i],tab_y[i])[0][0],corrdonnee_point(tab_x[i],0)[1][0]],[ corrdonnee_point(tab_x[i],tab_y[i])[0][1],corrdonnee_point(tab_x[i],0)[1][1]], c='r',label="solution1")
            plt.plot(corrdonnee_point(tab_x[i],0)[0][0],corrdonnee_point(tab_x[i],0)[0][1],'o',color='k',label="pivot Rotoide")
            plt.plot(P[0],P[1],'o',color='y',label="la pose donnée")
            plt.plot(tabX, tabY,'k-',label="l'espace atteignable")
            plt.plot(tabX, tabY1,'k-')
            plt.legend(loc='upper right',prop = {'size':11})
            plt.pause(0.001)
        
    
    if len(point_depart)==2 : #deux solutions possibles 
        
        [theta1,theta2],tab_x,tab_y=Newton(point_depart[0],precision,nmax)
        [theta1_prime,theta2_prime],tab_x_prime,tab_y_prime=Newton(point_depart[1],precision,nmax)
        for i in range(min(len(tab_x),len(tab_x_prime))):
            tabX = np.linspace(-l1-l2,l1+l2,2000)
            tabY = cercle1(tabX)
            tabY1= cercle2(tabX)
            plt.cla() #enlever l'inamge derniere
            plt.title("la position du bras articulé à différents instants avec le temps:10secondes\ndeux solutions possibles")
            plt.grid()
            plt.xlabel("x")
            plt.xlim(-l1-l2-1, l1+l2+1)
            plt.xticks(np.linspace(-l1-l2-5, l1+l2+5, 15, endpoint=True))
            plt.ylabel("y")
            plt.ylim(-l1-l2-1, l1+l2+1)
            plt.yticks(np.linspace(-l1-l2-5, l1+l2+5, 15, endpoint=True))
        
            plt.plot([0,corrdonnee_point(tab_x[i],tab_y[i])[0][0]],[0, corrdonnee_point(tab_x[i],tab_y[i])[0][1]], c='r')
            plt.plot([corrdonnee_point(tab_x[i],tab_y[i])[0][0],corrdonnee_point(tab_x[i],tab_y[i])[1][0]],[ corrdonnee_point(tab_x[i],tab_y[i])[0][1],corrdonnee_point(tab_x[i],tab_y[i])[1][1]], c='r',label="solution1")
            
            plt.plot([0,corrdonnee_point(tab_x_prime[i],tab_y_prime[i])[0][0]],[0, corrdonnee_point(tab_x_prime[i],tab_y_prime[i])[0][1]], c='b')
            plt.plot([corrdonnee_point(tab_x_prime[i],tab_y_prime[i])[0][0],corrdonnee_point(tab_x_prime[i],tab_y_prime[i])[1][0]],[ corrdonnee_point(tab_x_prime[i],tab_y_prime[i])[0][1],corrdonnee_point(tab_x_prime[i],tab_y_prime[i])[1][1]], c='b',label="solution2")
            
            plt.plot(corrdonnee_point(tab_x[i],0)[0][0],corrdonnee_point(tab_x[i],0)[0][1],'o',color='k',label="pivot Rotoide")
            plt.plot(corrdonnee_point(tab_x_prime[i],0)[0][0],corrdonnee_point(tab_x_prime[i],0)[0][1],'o',color='k')
            
            plt.plot(P[0],P[1],'o',color='y',label="la pose donnée")
            plt.plot(tabX, tabY,'k-',label="l'espace atteignable")#tracer l'espace atteignable
            plt.plot(tabX, tabY1,'k-')
            plt.legend(loc='upper right',prop = {'size':11})
            plt.pause(0.001)
       
    
    plt.ioff()
    plt.show()
    return

solution_et_trajectoire_avec_resultat_iteration([3,4],[2,3],1e-2,1000)

la methode de newton avec la précision choisi---> 0.01 et le nombre d'itération max---> 1000

veuillez patienter, les donneés en train de chargement, la compilation peut prendre jusqu'a une minute


veuillez taper 'yes' si vous vouler afficher les resultats de methode newton,sinon taper'no' NO




pour la pose---> [3, 4] et les longuer des bras---> [2, 3]   les resultats d'analyse finale par methode gradient sont:

le cas solution unique, point depart( [0.0, 0.0] )   --->theta: [0.927295, 0.0] 
vérification par modele geometrique: [3.0000008720063773, 3.999999345995068]
